In [ ]:
import os
import cv2
from torchvision.models import vgg16,VGG16_Weights
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from PIL import Image 
import torch
from tqdm.notebook import tqdm

In [ ]:
base_model = vgg16(weights=VGG16_Weights.DEFAULT)

feature_model = nn.Sequential(*list(base_model.children())[:-1]).to('cuda')

feature_model.eval()

In [ ]:
print(feature_model)

In [ ]:
root_dir = "dataset/"

image_dir = os.path.join(root_dir,"images")

In [ ]:
transformation = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
features = {}
for file_name in  tqdm(os.listdir(image_dir)):

    img_path = os.path.join(image_dir,file_name)  #Set the image path
    img = Image.open(img_path) #Read image
    if img.mode == 'L':
        img = img.convert('RGB')
    img = transformation(img) #apply trasnformation
    img = img.to('cuda')
    with torch.no_grad():     
        outputs = feature_model(img.unsqueeze(0))
    img_id = file_name.split('_')[2]
    img_id = int(img_id.split('.')[0])
    features[img_id] = outputs


In [ ]:
import pickle

In [ ]:
pickle.dump(features, open(os.path.join(root_dir, 'features.pkl'), 'wb'))


In [ ]:
with open('dataset/features.pkl','rb') as f:
    features = pickle.load(f)

In [ ]:
output = features[9]

In [ ]:
output.data

In [ ]:
processed = []
for feature_map in output:
    print(feature_map.shape)
    print(feature_map.shape)
    gray_scale = torch.sum(feature_map,0)
    print(gray_scale)
    gray_scale = gray_scale / feature_map.shape[0]
    processed.append(gray_scale.data.cpu().numpy())
for fm in processed:
    print(fm.shape)

In [ ]:
fig = plt.figure(figsize=(30, 50))
for i in range(len(processed)):
    a = fig.add_subplot(5, 4, i+1)
    imgplot = plt.imshow(processed[i])
    a.axis("off")
